In [1]:
!pip install optuna -q

In [178]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale


import matplotlib.pyplot as plt
import optuna

%matplotlib inline

In [179]:
x_test_mat100 = pd.read_csv('../data/Xte_mat100.csv',sep=' ',header=None).values
x_train_mat100 = pd.read_csv('../data/Xtr_mat100.csv',sep=' ',header=None).values


x_test = pd.read_csv('../data/Xte.csv',sep=',',index_col=0).values
x_train = pd.read_csv('../data/Xtr.csv',sep=',',index_col=0).values

y = pd.read_csv('../data/Ytr.csv',sep=',',index_col=0).values

In [180]:
print('x_train: {} y_train {}'.format(X_train_mat100.shape,y.shape))
print('x_train: {} y_train {}'.format(X_train.shape,y.shape))
print('x_test: {}'.format(X_test.shape))

x_train: (2000, 100) y_train (2000, 1)
x_train: (1340, 100) y_train (2000, 1)
x_test: (1000, 1)


In [181]:
X_train_mat100 = scale(X_train_mat100)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_mat100, y, test_size=0.33, random_state=42)

print(X_train.shape,X_val.shape,y_train.shape, y_val.shape)

(1340, 100) (660, 100) (1340, 1) (660, 1)


### Model Test

In [346]:

# Ridge Regression (RR)
def solveRR(y, X, lam):
    n, p = X.shape
    assert (len(y) == n)
    
    A = X.T.dot(X)
    # Adjust diagonal due to Ridge
    A[np.diag_indices_from(A)] += lam * n
    b = X.T.dot(y)
    
    # Hint:
    beta = np.linalg.solve(A, b)
    # Finds solution to the linear system Ax = b
    return (beta)

In [349]:
def cross_validate(x_data,y_data,lr,k=5):
    if len(x_data)%k != 0:
        print('cant vsplit',len(x_data),' by ',k)
        return
    
    x_data_splitted = np.vsplit(x_data,k)
    y_data_splitted = np.vsplit(y_data,k)
    
    aggrigate_result = []
    for i in range(len(x_data_splitted)):
        train = []
        test = []
        items = [j for j in range(len(x_data_splitted)) if j !=i ]
        x_test = x_data_splitted[i]
        y_test = y_data_splitted[i]
        for item in items:
            if len(train) == 0:
                x_train = x_data_splitted[item]
                y_train = y_data_splitted[item]
            else:
                x_train = np.concatenate((x_train,x_data_splitted[item]), axis=0)
                y_train = np.concatenate((y_train,y_data_splitted[item]), axis=0)
            
        logistic = LogisticRegression(lr=0.01, num_iter=100000, fit_intercept=True)
        logistic.fit(x_train,y_train)
        
        result = accuracy_score(logistic.predict(x_test),y_test)
        aggrigate_result.append(result)
        
    return sum(aggrigate_result)/len(aggrigate_result)

In [350]:
cross_validate(X_train_mat100, y,0.001,10)

ValueError: operands could not be broadcast together with shapes (101,) (101,200) (101,) 